In [4]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)



In [5]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()):
    if "anuja" in os.environ.get('USER'):
        DATA_DIR = 'data/'
    elif 'ubuntu' in os.environ.get('USER'):
        DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/DATAfoof'
    

## Data Loading and PCA

In [6]:
path = os.path.join(DATA_DIR, 'split_data_csv')

#loading
xtrain = pd.read_csv(os.path.join(path, 'train_features_sensor.csv')).drop('IDs', axis = 1)
xtest = pd.read_csv(os.path.join(path, 'test_features_sensor_regression.csv')).drop('IDs', axis = 1)
ytrain = pd.read_csv(os.path.join(path, 'train_labels_regression.csv')).drop('IDs', axis = 1)
ytest = pd.read_csv(os.path.join(path, 'test_labels_regression.csv')).drop('IDs', axis = 1)


In [7]:
# dealing with NaNs
train_set = pd.concat([xtrain, ytrain], axis=1)
train_set = train_set.dropna()

labels_list = ['SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg', 'SCARED_P_GD','WISC_WMI_Sum', 'WISC_VCI_Sum']

xtrain = pd.DataFrame.copy(train_set)
xtrain = xtrain.drop(labels_list, axis=1)

ytrain = train_set[labels_list]


In [8]:
xtrain = np.array(xtrain)
xtest = np.array(xtest)
ytrain = np.array(ytrain) 
ytest = np.array(ytest) 

In [9]:
age_gender = True
if age_gender:
    train_age_gender = xtrain[:,-2:]
    test_age_gender = xtest[:,-2:]
    xtrain = xtrain[:,:-2]
    xtest = xtest[:,:-2]

In [10]:
# scaling x
norm = preprocessing.MinMaxScaler().fit(xtrain)

# transform training data
xtrain = norm.transform(xtrain)
xtest = norm.transform(xtest)
print(xtrain.shape, xtest.shape)

print('Applying PCA...')
pca = PCA(.95) # 95% variance retained
pca.fit(xtrain)

# transform data
xtrain = pca.transform(xtrain)
xtest = pca.transform(xtest)
print(xtrain.shape, xtest.shape)

(795, 8505) (275, 8505)
Applying PCA...
(795, 198) (275, 198)


In [11]:
if age_gender:
    xtrain = np.concatenate([xtrain, train_age_gender], axis = 1)
    xtest = np.concatenate([xtest, test_age_gender], axis = 1)

In [12]:
# scaling y
min_max_scaler = preprocessing.MinMaxScaler().fit(ytrain)
ytrain = min_max_scaler.transform(ytrain)
ytest = min_max_scaler.transform(ytest)
print(ytrain.shape, ytest.shape)

(795, 7) (275, 7)


In [13]:
ytrain.mean(axis=0)

array([0.40676397, 0.33289693, 0.61721403, 0.54358255, 0.25988819,
       0.48893082, 0.49404366])

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [14]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor


lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))


Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.13
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.01


In [15]:
print(r2_score(ytest, ypred_dummy_mean, multioutput='raw_values'))
print(r2_score(ytest, ypred_dummy_median, multioutput='raw_values'))


[-1.10961530e-02 -2.73451274e-03 -7.17039586e-04 -6.30590131e-04
 -6.35375772e-04 -3.30013012e-03 -8.66833499e-05]
[-6.59384160e-04 -4.22098973e-02 -1.56859084e-03 -6.77862253e-05
 -1.96013461e-02 -1.44334999e-02 -2.19895313e-02]


In [16]:
# save predictions to csv file
ypred = np.array(ypred_dummy_mean)
np.savetxt('predicted_labels_reg_dummy_mean.csv', min_max_scaler.inverse_transform(ypred), delimiter=',')

## MOR regressor with base SVR regressor

In [17]:
n = 2

In [18]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils import resample

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
model = MultiOutputRegressor(svr)
# Train the regressor
#model = model.fit(xtrain, ytrain)
# Generate predictions for testing data
#ypred = model.predict(xtest)


mse = []
r = []
mae = []
rm = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	model = model.fit(xtrain, ytrain)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))
	rm.append(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

print("Mean squared error (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (SVR) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))




Mean squared error (SVR): 0.05 [0.05, 0.05]
Mean abs error (SVR) 0.18 [0.18, 0.18]
R2 score (SVR): -0.08 [-0.09, -0.07]


In [19]:
rma = np.array(rm)
r=rma[:,0]


for i in range(7):
    r = rma[:,i]
    print("R2 score {:.3f} [{:.3f}, {:.3f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))


R2 score -0.074 [-0.122, -0.026]
R2 score -0.026 [-0.063, 0.011]
R2 score -0.167 [-0.189, -0.145]
R2 score -0.027 [-0.048, -0.006]
R2 score -0.199 [-0.229, -0.168]
R2 score -0.038 [-0.067, -0.008]
R2 score -0.028 [-0.062, 0.007]


In [20]:
print(model.score(resampled_xtest, resampled_ytest))
print(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

#np.mean(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

-0.06476631907617059
[-0.02067456  0.01489632 -0.14262278 -0.00313306 -0.16503105 -0.07046924
 -0.06632986]


In [21]:
# save predictions to csv file
ypred = np.array(ypred)

np.savetxt('predicted_labels_reg_MOR_SVR.csv', min_max_scaler.inverse_transform(ypred), delimiter=',')

In [22]:
%%capture

import sys, os

predicted_labels_shuffled = np.copy(ytest)
np.random.shuffle(predicted_labels_shuffled)

print("r2_score (labels_shuffled): {:.2f}".format(r2_score(ytest, predicted_labels_shuffled)))


r = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,predicted_labels_shuffled))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,predicted_labels_shuffled))


print("R2 score (labels_shuffled): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))
print(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

In [23]:
# save predictions to csv file
ypred = np.array(ypred)
np.savetxt('predicted_labels_reg_shuffled.csv', min_max_scaler.inverse_transform(ypred), delimiter=',')

## MOR RandomForestRegressor

In [24]:
max_depth = 30
model = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	model = model.fit(xtrain, ytrain)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))
	rm.append(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

print("Mean squared error (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (RanFor) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

Mean squared error (RanFor): 0.05 [0.05, 0.05]
Mean abs error (RanFor) 0.17 [0.17, 0.17]
R2 score (RanFor): -0.02 [-0.02, -0.01]


In [25]:
rma = np.array(rm)
r=rma[:,0]


for i in range(7):
    r = rma[:,i]
    print("R2 score {:.3f} [{:.3f}, {:.3f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

R2 score -0.051 [-0.115, -0.011]
R2 score -0.009 [-0.058, 0.025]
R2 score -0.092 [-0.184, -0.008]
R2 score 0.012 [-0.044, 0.052]
R2 score -0.127 [-0.222, -0.049]
R2 score -0.059 [-0.081, -0.015]
R2 score -0.015 [-0.061, 0.026]


In [26]:
print(r2_score(resampled_ytest, ypred, multioutput='raw_values'))


[-0.00875685  0.02667263 -0.00491797  0.05277731 -0.04698809 -0.07665048
 -0.0328002 ]


In [27]:
# save predictions to csv file
ypred = np.array(ypred)
np.savetxt('predicted_labels_reg_MOR_randfor.csv', min_max_scaler.inverse_transform(ypred), delimiter=',')

## CHAINS

In [28]:
# chained multioutput regressor
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import RandomForestRegressor
# define dataset
# define base model
model = RandomForestRegressor()
# define the chained multioutput wrapper model
model = RegressorChain(model)
# Train the regressor

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	model = model.fit(xtrain, ytrain)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))
	rm.append(r2_score(resampled_ytest, ypred, multioutput='raw_values'))
	
print("Mean squared error (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (RanFor) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

In [ ]:
rma = np.array(rm)
r=rma[:,0]


for i in range(7):
    r = rma[:,i]
    print("R2 score {:.3f} [{:.3f}, {:.3f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

R2 score 0.004 [-0.019, 0.033]
R2 score -0.028 [-0.070, 0.049]
R2 score -0.066 [-0.141, -0.025]
R2 score 0.012 [-0.067, 0.057]
R2 score -0.066 [-0.087, -0.050]
R2 score -0.066 [-0.176, 0.014]
R2 score -0.068 [-0.163, 0.013]


In [ ]:
print(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

[-0.00103813 -0.0692927  -0.05094765  0.0074613  -0.07471735  0.00542172
  0.01573559]


In [ ]:
# save predictions to csv file
ypred = np.array(ypred)
np.savetxt('predicted_labels_reg_chain_randfor.csv', min_max_scaler.inverse_transform(ypred), delimiter=',')

In [ ]:
# chained multioutput regressor
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import SVR
# define dataset
# define base model
model = SVR()
# define the chained multioutput wrapper model
model = RegressorChain(model)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	# Train the regressor
	model = model.fit(xtrain, ytrain)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))
	rm.append(r2_score(resampled_ytest, ypred, multioutput='raw_values'))

print("Mean squared error (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (SVR) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

Mean squared error (SVR): 0.05 [0.05, 0.05]
Mean abs error (SVR) 0.17 [0.17, 0.18]
R2 score (SVR): -0.09 [-0.11, -0.08]


In [ ]:
rma = np.array(rm)
r=rma[:,0]


for i in range(7):
    r = rma[:,i]
    print("R2 score {:.3f} [{:.3f}, {:.3f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

R2 score 0.006 [-0.019, 0.033]
R2 score -0.021 [-0.070, 0.048]
R2 score -0.094 [-0.180, -0.025]
R2 score -0.006 [-0.094, 0.057]
R2 score -0.079 [-0.132, -0.051]
R2 score -0.086 [-0.183, 0.013]
R2 score -0.091 [-0.165, 0.012]


In [ ]:
print(r2_score(resampled_ytest, ypred, multioutput='raw_values'))


[-0.00776625 -0.01304241 -0.18439903 -0.10109851 -0.15489796 -0.12548793
 -0.16215264]


In [ ]:
# save predictions to csv file
ypred = np.array(ypred)
np.savetxt('predicted_labels_reg_chain_svr.csv', min_max_scaler.inverse_transform(ypred), delimiter=',')